In [1]:
import numpy as np
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup as bs
#from google.colab import files
from tqdm import *
#courses = pd.read_csv('courses_all.csv')
#career360_courses = pd.DataFrame(columns=['College','City','State','Course','Duration','Study mode','Approved intake','Course level','Course Fee','Approval','Eligiblity','Course_Detail'])

In [2]:
eng_list = []

In [3]:
for i in tqdm(range(34)):
    page = rq.get('https://law.careers360.com/colleges/list-of-law-colleges-in-india?page='+str(i))
    soup = bs(page.content,'html.parser')
    vals = soup.find_all('div',class_='title')
    for val in vals:
        try:
            eng_list.append(val.a['href'])
        except:
            pass

100%|██████████| 34/34 [00:36<00:00,  1.08s/it]


In [4]:
pd.DataFrame(eng_list).to_csv('mang_list.csv')

In [5]:
courses = pd.DataFrame(columns=[0,1,2])

In [6]:
idx = 0
for link in tqdm(eng_list):
    try:
        page = rq.get(link)
        soup = bs(page.content,'html.parser')
        courses.loc[idx]=[link,soup.find_all('h4',class_='blockSubHeading')[0].get_text()[-2:],'https://law.careers360.com/'+soup.find_all('ul',class_='list-tabs')[0].find_all('li')[1].a['href']]
        idx+=1
    except:
        print(f'error in {idx} and {link}')
        pass

 25%|██▌       | 254/1015 [02:12<06:36,  1.92it/s]

error in 253 and https://law.careers360.com/colleges/school-of-law-christ-university-bangalore


100%|██████████| 1015/1015 [07:59<00:00,  2.12it/s]


(1014, 3)

In [8]:
courses.to_csv('law_courses.csv')

In [9]:
from threading import Thread
def scrapper(start, end):
    career360_courses = pd.DataFrame(columns=['College','City','State','Course','Duration','Study mode','Approved intake','Course level','Course Fee','Approval','Eligiblity','Course_Detail'])
    indx = 0
    cnt = start - 1
    for header_course in tqdm(courses[2][start:end]):
        cnt += 1
        try:
            num = int(np.ceil(int(courses[1][cnt])/10))
        except:
            continue
    #     print(f'num = {num}')
        for page_num in tqdm(range(num)):
            link = header_course + '?levelId=all&courseNid=all-all&page='+str(page_num)
    #         print(link)
            page = rq.get(link)
            course_soup = bs(page.content, 'html.parser')
        for course_link in course_soup.find_all('span',class_='readmore'):
            course_link = 'https://law.careers360.com/' + course_link.a['href']
            #             print(f'course_link = {course_link}')
            page1 = rq.get(course_link)
            soup = bs(page1.content,'html.parser')
            career360_courses.loc[indx]=None

            try:
                career360_courses.loc[indx]['Course'] = soup.find_all('h2',class_='block-title')[0].get_text()
                career360_courses.loc[indx]['College']=soup.find_all('h1',class_='titleNameCol')[0].get_text()
                career360_courses.loc[indx]['City'] = soup.find_all('ul',class_='clg-info')[0].find_all('li')[0].find_all('a')[0].get_text()
                career360_courses.loc[indx]['State'] = soup.find_all('ul',class_='clg-info')[0].find_all('li')[0].find_all('a')[1].get_text()
            except:
                pass

            try:
                career360_courses.loc[indx]['Eligiblity'] = soup.find_all('span',class_='more-eligibility')[0].get_text()[:-11]
            except:
                try:
                    career360_courses.loc[indx]['Eligiblity'] = soup.find_all('div',class_='default-elig')[0].get_text()
                except:
                    career360_courses.loc[indx]['Eligiblity'] = None

            try:
                career360_courses.loc[indx]['Course_Detail'] = (soup.find_all('span',class_='moreCourseDetails')[0].p.get_text())[:-11]
            except:
                try:
                    career360_courses.loc[indx]['Course_Detail'] = (soup.find_all('div',class_='default-course-details')[0].get_text())
                except:
                    career360_courses.loc[indx]['Course_Detail'] = None

            for row in soup.find_all('div',class_='coursesPageLableInnerSec'):
                career360_courses.loc[indx][row.strong.get_text()] = row.p.get_text()
            indx+=1
    career360_courses.to_csv(f'career360_scrap_dm_law({start}-{end}).csv')

In [10]:
siz = len(eng_list)
numer = int(siz/20)
prev = 0
def threader():
    threads = []
    for i in range(20):
        if i!=19:
            thread = Thread(target=scrapper, args = (i*numer,(i+1)*numer))
            threads.append(thread)
            thread.start()
        else:
            thread = Thread(target=scrapper, args = (19*numer,siz))
            threads.append(thread)
            thread.start()        
    
    for thread in threads:  # iterates over the threads
        thread.join()
                    
    print('done')

In [11]:
threader()

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]


  0%|          | 0/1 [00:00<?, ?it/s]



  0%|          | 0/50 [00:00<?, ?it/s]




  0%|          | 0/1 [00:00<?, ?it/s]





  0%|          | 0/50 [00:00<?, ?it/s]






  0%|          | 0/1 [00:00<?, ?it/s]







  0%|          | 0/50 [00:00<?, ?it/s]








  0%|          | 0/1 [00:00<?, ?it/s]









  0%|          | 0/50 [00:00<?, ?it/s]










  0%|          | 0/1 [00:00<?, ?it/s]











  0%|          | 0/50 [00:00<?, ?it/s]












  0%|          | 0/1 [00:00<?, ?it/s]













  0%|          | 0/50 [00:00<?, ?it/s]














  0%|          | 0/1 [00:00<?, ?it/s]















  0%|          | 0/50 [00:00<?, ?it/s]
















  0%|          | 0/1 [00:00<?, ?it/s]


















  0%|          | 0/50 [00:00<?, ?it/s]



















  0%|          | 0/1 [00:00<?, ?it/s]




















  0%|          | 0/50 [00:00<?, ?it/s]





















  0%|          |

  2%|▏         | 1/50 [00:05<04:32,  5.57s/it]





















  0%|          | 0/1 [00:00<?, ?it/s]

  2%|▏         | 1/50 [00:06<05:17,  6.49s/it]






















  0%|          | 0/1 [00:00<?, ?it/s]








  6%|▌         | 3/50 [00:06<01:45,  2.24s/it]























  0%|          | 0/1 [00:00<?, ?it/s]


















  2%|▏         | 1/50 [00:06<05:29,  6.73s/it]
























  0%|          | 0/1 [00:00<?, ?it/s]







  2%|▏         | 1/50 [00:07<05:52,  7.18s/it]

























  0%|          | 0/1 [00:00<?, ?it/s]












  2%|▏         | 1/50 [00:07<05:53,  7.22s/it]


























  0%|          | 0/1 [00:00<?, ?it/s]





  2%|▏         | 1/50 [00:07<06:02,  7.40s/it]



























  0%|          | 0/1 [00:00<?, ?it/s]









  2%|▏         | 1/50 [00:07<06:01,  7.37s/it]




























  2%|▏         | 1/50 [00:07<06:11,  7.58s/it]





























  0%|          | 0/1 [00:0

  2%|▏         | 1/50 [00:13<11:00, 13.49s/it]


























  0%|          | 0/1 [00:00<?, ?it/s]





















100%|██████████| 1/1 [00:00<00:00,  2.03it/s]













































100%|██████████| 1/1 [00:00<00:00,  2.11it/s]











































100%|██████████| 1/1 [00:02<00:00,  2.65s/it]





























  4%|▍         | 2/50 [00:15<06:02,  7.55s/it]
























  0%|          | 0/1 [00:00<?, ?it/s]



















100%|██████████| 1/1 [00:02<00:00,  2.18s/it]










































100%|██████████| 1/1 [00:02<00:00,  2.12s/it]










































100%|██████████| 1/1 [00:02<00:00,  2.38s/it]






























  4%|▍         | 2/50 [00:15<06:21,  7.96s/it]






















  0%|          | 0/1 [00:00<?, ?it/s]



















100%|██████████| 1/1 [00:03<00:00,  3.21s/it]







































100%|██████████| 1/1 [

  6%|▌         | 3/50 [00:22<05:56,  7.58s/it]


























  0%|          | 0/1 [00:00<?, ?it/s]














  6%|▌         | 3/50 [00:22<05:54,  7.55s/it]



























  0%|          | 0/1 [00:00<?, ?it/s]


  4%|▍         | 2/50 [00:23<09:26, 11.80s/it]




























  0%|          | 0/1 [00:00<?, ?it/s]



















100%|██████████| 1/1 [00:01<00:00,  1.85s/it]




































  6%|▌         | 3/50 [00:23<06:09,  7.86s/it]




























  0%|          | 0/1 [00:00<?, ?it/s]



















100%|██████████| 1/1 [00:02<00:00,  2.11s/it]








































100%|██████████| 1/1 [00:02<00:00,  2.23s/it]








































100%|██████████| 1/1 [00:02<00:00,  2.51s/it]








































100%|██████████| 1/1 [00:02<00:00,  2.28s/it]









































100%|██████████| 1/1 [00:02<00:00,  2.83s/it]


























100%|██████████| 1/1 [00:04<00:00,  4.04s/it]






























 12%|█▏        | 6/50 [00:33<04:06,  5.61s/it]
























  0%|          | 0/1 [00:00<?, ?it/s]
















  8%|▊         | 4/50 [00:33<06:25,  8.39s/it]

























  0%|          | 0/1 [00:00<?, ?it/s]




  4%|▍         | 2/50 [00:33<13:32, 16.93s/it]


























  0%|          | 0/1 [00:00<?, ?it/s]




















100%|██████████| 1/1 [00:04<00:00,  5.00s/it]


































  8%|▊         | 4/50 [00:33<06:28,  8.45s/it]


























  0%|          | 0/1 [00:00<?, ?it/s]





















100%|██████████| 1/1 [00:02<00:00,  2.17s/it]










































100%|██████████| 1/1 [00:03<00:00,  3.67s/it]









































100%|██████████| 1/1 [00:03<00:00,  3.17s/it]










































100%|██████████| 1/1 [00:02<00:00,  2.82s/it]































 1

...





 80%|████████  | 40/50 [07:20<01:50, 11.02s/it]
















  0%|          | 0/1 [00:00<?, ?it/s]


 74%|███████▍  | 37/50 [07:21<02:34, 11.92s/it]

















  0%|          | 0/1 [00:00<?, ?it/s]










 92%|█████████▏| 59/64 [07:20<00:37,  7.47s/it]


















  0%|          | 0/1 [00:00<?, ?it/s]












100%|██████████| 1/1 [00:02<00:00,  2.44s/it]














 70%|███████   | 35/50 [07:21<03:09, 12.62s/it]


















  0%|          | 0/1 [00:00<?, ?it/s]












100%|██████████| 1/1 [00:01<00:00,  1.73s/it]

























100%|██████████| 1/1 [00:01<00:00,  1.55s/it]

















 78%|███████▊  | 39/50 [07:22<02:04, 11.36s/it]

















  0%|          | 0/1 [00:00<?, ?it/s]
















100%|██████████| 1/1 [00:01<00:00,  1.58s/it]






























100%|██████████| 1/1 [00:02<00:00,  2.43s/it]



























100%|██████████| 1/1 [00:02<00:00,  2.43s/it]



























100%|███████

 76%|███████▌  | 38/50 [07:39<02:24, 12.08s/it]















  0%|          | 0/1 [00:00<?, ?it/s]












100%|██████████| 1/1 [00:02<00:00,  2.22s/it]























 98%|█████████▊| 63/64 [07:39<00:07,  7.29s/it]















  0%|          | 0/1 [00:00<?, ?it/s]












100%|██████████| 1/1 [00:02<00:00,  2.11s/it]

























100%|██████████| 1/1 [00:02<00:00,  2.01s/it]

























100%|██████████| 1/1 [00:02<00:00,  2.02s/it]

























100%|██████████| 1/1 [00:01<00:00,  1.70s/it]























100%|██████████| 64/64 [07:42<00:00,  7.23s/it]












 78%|███████▊  | 39/50 [07:43<02:10, 11.88s/it]











  0%|          | 0/1 [00:00<?, ?it/s]


 80%|████████  | 40/50 [07:43<01:55, 11.59s/it]












  0%|          | 0/1 [00:00<?, ?it/s]






 92%|█████████▏| 46/50 [07:43<00:40, 10.09s/it]













  0%|          | 0/1 [00:00<?, ?it/s]




 82%|████████▏ | 41/50 [07:44<01:41, 11.33s/it]











100%|██████████| 1/1 [00:01<00:00,  1.92s/it]






















100%|██████████| 8/8 [00:15<00:00,  1.91s/it]












 82%|████████▏ | 41/50 [08:04<01:46, 11.83s/it]











  0%|          | 0/1 [00:00<?, ?it/s]




 86%|████████▌ | 43/50 [08:04<01:18, 11.28s/it]












  0%|          | 0/1 [00:00<?, ?it/s]










100%|██████████| 1/1 [00:01<00:00,  1.77s/it]


















100%|██████████| 50/50 [08:06<00:00,  9.73s/it]







 82%|████████▏ | 41/50 [08:06<01:46, 11.87s/it]











  0%|          | 0/1 [00:00<?, ?it/s]





 86%|████████▌ | 43/50 [08:06<01:19, 11.32s/it]












  0%|          | 0/1 [00:00<?, ?it/s]










100%|██████████| 1/1 [00:01<00:00,  1.82s/it]




















100%|██████████| 1/1 [00:02<00:00,  2.03s/it]




















100%|██████████| 1/1 [00:01<00:00,  1.79s/it]




















100%|██████████| 1/1 [00:01<00:00,  1.91s/it]













 90%|█████████ | 45/50 [08:08<00:54, 10.87s/it]









  0%|          | 0/1 [

 90%|█████████ | 45/50 [08:36<00:57, 11.47s/it][A










  0%|          | 0/1 [00:00<?, ?it/s]





 98%|█████████▊| 49/50 [08:36<00:10, 10.54s/it]











  0%|          | 0/1 [00:00<?, ?it/s]



 98%|█████████▊| 49/50 [08:37<00:10, 10.56s/it]












  0%|          | 0/1 [00:00<?, ?it/s]








100%|██████████| 1/1 [00:01<00:00,  1.99s/it]


















100%|██████████| 1/1 [00:01<00:00,  1.71s/it]


















100%|██████████| 1/1 [00:02<00:00,  2.30s/it]

















100%|██████████| 1/1 [00:02<00:00,  2.46s/it]

















100%|██████████| 1/1 [00:01<00:00,  1.58s/it]










 92%|█████████▏| 46/50 [08:39<00:45, 11.30s/it]








  0%|          | 0/3 [00:00<?, ?it/s]



100%|██████████| 50/50 [08:41<00:00, 10.43s/it]







100%|██████████| 50/50 [08:41<00:00, 10.43s/it]






100%|██████████| 50/50 [08:41<00:00, 10.44s/it]








 92%|█████████▏| 46/50 [08:42<00:45, 11.37s/it]A






  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 50/50 [

done


In [12]:
scrapper(50,100)




  0%|          | 0/50 [00:00<?, ?it/s]



  0%|          | 0/1 [00:00<?, ?it/s]



100%|██████████| 1/1 [00:00<00:00,  2.59it/s]






  2%|▏         | 1/50 [00:02<02:04,  2.53s/it]



  0%|          | 0/1 [00:00<?, ?it/s]



100%|██████████| 1/1 [00:00<00:00,  2.28it/s]






  4%|▍         | 2/50 [00:03<01:33,  1.94s/it]



  0%|          | 0/1 [00:00<?, ?it/s]



100%|██████████| 1/1 [00:00<00:00,  2.54it/s]






  6%|▌         | 3/50 [00:06<01:47,  2.29s/it]



  0%|          | 0/2 [00:00<?, ?it/s]



 50%|█████     | 1/2 [00:00<00:00,  2.19it/s]



100%|██████████| 2/2 [00:00<00:00,  2.14it/s]






  8%|▊         | 4/50 [00:09<01:47,  2.34s/it]



  0%|          | 0/1 [00:00<?, ?it/s]



100%|██████████| 1/1 [00:00<00:00,  2.32it/s]






 10%|█         | 5/50 [00:10<01:32,  2.05s/it]



  0%|          | 0/10 [00:00<?, ?it/s]



 10%|█         | 1/10 [00:00<00:04,  2.08it/s]



 20%|██        | 2/10 [00:01<00:04,  1.87it/s]



 30%|███       | 3/10 [00:01<00:03,  2.06it/s]




  0%|          | 0/1 [00:00<?, ?it/s]



100%|██████████| 1/1 [00:02<00:00,  2.11s/it]






 78%|███████▊  | 39/50 [02:55<00:49,  4.50s/it]



  0%|          | 0/1 [00:00<?, ?it/s]



100%|██████████| 1/1 [00:01<00:00,  1.64s/it]






 80%|████████  | 40/50 [03:02<00:45,  4.56s/it]



  0%|          | 0/1 [00:00<?, ?it/s]



100%|██████████| 1/1 [00:01<00:00,  1.46s/it]






 82%|████████▏ | 41/50 [03:23<00:44,  4.97s/it]



  0%|          | 0/1 [00:00<?, ?it/s]



100%|██████████| 1/1 [00:01<00:00,  1.67s/it]






 84%|████████▍ | 42/50 [03:39<00:41,  5.22s/it]



  0%|          | 0/1 [00:00<?, ?it/s]



100%|██████████| 1/1 [00:01<00:00,  1.54s/it]






 86%|████████▌ | 43/50 [03:49<00:37,  5.33s/it]



  0%|          | 0/1 [00:00<?, ?it/s]



100%|██████████| 1/1 [00:01<00:00,  1.47s/it]






 88%|████████▊ | 44/50 [03:52<00:31,  5.29s/it]



  0%|          | 0/1 [00:00<?, ?it/s]



100%|██████████| 1/1 [00:01<00:00,  1.53s/it]






 90%|█████████ | 45/50 [04:05<00:27,  5.46s